
# <b>Offline Video Service Demo</b>


The AIServiceVisionClient offers the video features. This notebook aims to provide overall clarity about the features to the user in terms of requirements, usage.
<ul>
    <li><font size="2">The output response file is stored at the object storage location specified in the below cells.</font></li>
    <li><font size="2">At the end of the notebook the response is downloaded and saved as <code>video_response.json</code> in the output directory. </font></li>
</ul>


### Steps to run the notebook:
<details>
    <summary>Notebook session setup</summary>
    <ol>
        <li><font size="2">Installing the OCI SDK</font></li>
        <li><font size="2">Installing other dependencies</font></li>
        <li><font size="2">Setup sample input images</font></li>
        <li><font size="2">Create output folder</font></li>
        <li><font size="2">Setup helper .py files</font></li>
    </ol>
</details>

<details>
    <summary>Importing the required modules</summary>
    <ol>
    </ol>
</details>

<details>
    <summary>Setting the input variables</summary>
    <ol>
        <li><font size="2">The user can give input variables of their choice.</font></li>
    </ol>
</details>

<details>
    <summary>Running the main pipeline</summary>
    <ol>
        <li><font size="2">Run all cells to get the output in the <code>output</code> directory. </font><br></li>
    </ol>
</details>

### Notebook session setup
<details>
    <summary>Instructions</summary>
    <ul>
        <li><font size="2">The user needs to setup only once.</font></li>
        <li><font size="2">Uncomment the commented cells and run once to setup.</font></li>
        <li><font size="2">Comment back the same cells to avoid running again.</font></li>
    </ul>
</details>

#### Installing the OCI Python SDK
Contact OCI AI Vision team to get access to preview SDK. Download and run the below command to install python SDK.

In [ ]:
# Please use pip/pip3 as per availability
# !pip3 install 2.115.2+preview.1.1678-py2.py3-none-any.whl

#### Setup sample input

* Uncomment and run the cell below.
* Create a bucket in your tenancy (you may skip this step if you have an existing bucket)
* Upload the video `demo.mp4` to the bucket. (you can also upload and use the video of your choice)

In [ ]:
# !wget "https://github.com/oracle-samples/oci-data-science-ai-samples/tree/main/labs/ai-vision/analyze_video_workshop/data/demo.mp4"
# !mkdir data
# !mv demo.mp4 data

#### Setup helper.py files

In [ ]:
# !wget "https://github.com/oracle-samples/oci-data-science-ai-samples/tree/main/labs/ai-vision/analyze_video_workshop/helper/analyze_video_utils.py"
# !mkdir helper
# !mv analyze_video_utils.py helper

#### Import Libraries

In [ ]:
import time
import oci
from oci.ai_vision.models import *
from helper.analyze_video_utils import display_formatted_response
from IPython.display import JSON

#### Authorize OCI config
Set up authentication for OCI by reading configuration from a file. The default configuration file location is ```~/.oci/config``` and ```DEFAULT``` profile is choosen.

In [ ]:
config = oci.config.from_file('~/.oci/config', profile_name='DEFAULT')

#### Set required input variables
<details>
    <summary><font size="3">input_location_variables</font></summary>
    <font size="2">The user needs to provide the following details:
        <ul>
            <li><code>namespace</code> : specify the namespace where the input video is uploaded</li>
            <li><code>bucket</code> : specify the bucket name where the input video is uploaded</li>
            <li><code>filename</code> : specify the filename of the input video(e.g: we have uploaded <code>demo.mp4</code> in <code>Setup sample input</code> step )</li>
        </ul>
    </font>
</details>

<details>
    <summary><font size="3">output_location_path</font></summary>
    <font size="2">The user needs to provide the following details to store the output:
        <ul>
            <li><code>namespace</code> : specify the namespace where the output has to be stored</li>
            <li><code>bucket</code> : specify the bucket name where the output has to be stored</li>
            <li><code>prefix</code> : specify the prefix where the output has to be stored</li>
        </ul>
    </font>
</details>

<details>
<summary><font size="3">compartment_id</font></summary>
    <font size="2">The user should provide the compartment OCID to call the API.</font><br>
</details>

In [ ]:
input_namespace = "<INPUT_NAMESPACE>"
input_bucket = "<INPUT_BUCKET>"
input_filename = "<INPUT_FILENAME>"

output_namespace = "<OUTPUT_NAMESPACE>"
output_bucket = "<OUTPUT_BUCKET>"
output_prefix = "<OUTPUT_PREFIX>"

compartment_id = "<COMPARTMENT_ID>"

#### Set optional input variables

<details>
<summary><font size="3">max_results</font></summary>
    <font size="2">Provide the maximum results needed for Object, Label and Face Detection. This is an upper limit over the output classes, the API may detect lesser classes according to the input video. In case of Object and Label Detection default value is 5. Whereas for Face Detection default value is 50 </font><br>
</details>

<details>
<summary><font size="3">min_confidence</font></summary>
    <font size="2">Provide the minimum confidence needed for the feature. This is an lower limit over the output, the API may detect objects and classes above the specified confidence. Default value is 0</font><br>
</details>

<details>
<summary><font size="3">model_id</font></summary>
    <font size="2">In case of custom models uncomment the line and provide the model_id needed for the feature.</font><br>
</details>

In [ ]:
max_results = 5

min_confidence = 0 

# model_id = "<MODEL_ID>" 

#### Setup input location

In [ ]:
object_location_1 = ObjectLocation()
object_location_1.namespace_name = input_namespace
object_location_1.bucket_name = input_bucket
object_location_1.object_name = input_filename
object_locations = [object_location_1]
input_location = ObjectListInlineInputLocation()
input_location.object_locations = object_locations

#### Setup output location

In [ ]:
output_location = OutputLocation()
output_location.namespace_name = output_namespace
output_location.bucket_name = output_bucket
output_location.prefix = output_prefix

### Create AIServiceVisionClient and Setup input feature for Offline video features
You can specify the features you want to call. In the below code all supported pretrained models are called. To call custom models uncomment commented lines and replace your model id

In [ ]:
ai_service_vision_client = oci.ai_vision.AIServiceVisionClient(config=config)

video_label_detection_feature = VideoLabelDetectionFeature()
video_label_detection_feature.max_results = max_results
video_label_detection_feature.min_confidence = min_confidence
# video_label_detection_feature.model_id = model_id

video_object_detection_feature = VideoObjectDetectionFeature()
video_object_detection_feature.max_results = max_results
video_object_detection_feature.min_confidence = min_confidence
# video_object_detection_feature.model_id = model_id

video_text_detection_feature = VideoTextDetectionFeature()
video_text_detection_feature.min_confidence = min_confidence

video_face_detection_feature = VideoFaceDetectionFeature()
video_face_detection_feature.max_results = max_results
video_face_detection_feature.min_confidence = min_confidence

features = [video_label_detection_feature, 
            video_object_detection_feature, 
            video_text_detection_feature, 
            video_face_detection_feature]

### Create video job

In [ ]:
create_video_job_details = CreateVideoJobDetails()
create_video_job_details.features = features
create_video_job_details.compartment_id = compartment_id
create_video_job_details.output_location = output_location
create_video_job_details.input_location = input_location

res = ai_service_vision_client.create_video_job(create_video_job_details=create_video_job_details)


### Job submitted
The job is created and should be in <code>ACCEPTED</code> state.

In [ ]:
JSON(display_formatted_response(res))

#### Job IN_PROGRESS
The job progress is tracked till completion with a polling interval of 5 seconds.

In [ ]:
job_id = res.data.id
polling_interval = 5
print("Job ID :", job_id, '\n')
seconds = 0
while res.data.lifecycle_state == "IN_PROGRESS" or res.data.lifecycle_state == "ACCEPTED":
    print(f"Job {job_id} is IN_PROGRESS for {str(seconds)} seconds, progress: {res.data.percent_complete}")
    time.sleep(polling_interval)
    seconds += polling_interval
    res = ai_service_vision_client.get_video_job(video_job_id=job_id)

### Job completed
The job is completed and should be in <code>SUCCEEDED</code> state. In case the job is not in <code>SUCCEEDED</code> state refer to the displayed response below to know the state and reason.

In [ ]:
JSON(display_formatted_response(res))

#### Get response json from object storage
The output can be found in the output location specified or it can be saved in ```video_response.json``` file by running the below cell

In [ ]:
object_storage_client = oci.object_storage.ObjectStorageClient(config)
output_object_name = f"{output_location.prefix}/{job_id}/{input_location.object_name}.json"

video_response = object_storage_client.get_object(output_location.namespace_name, output_location.bucket_name, output_object_name)

file = open('video_response.json', 'w')
file.write(video_response.data.text)